In [90]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import log_loss

In [134]:
class LogisticRegressionClassifier:
    def __init__(self, degree=2, max_iter=10000, tol=0.001):
        self.degree = degree
        self.max_iter = max_iter
        self.tol = tol
        self.weights = None
        self.classes = None
        self.cls_one_hot_encoding = None  # ground truth

    def predict(self):
        # Sigmoid function
        pass

    def fit(self, X, y):
        # Gradient Descent
        # transform data in degree 2 space: 1, a, b, a**2, a*b, b**2
        if self.degree == 2: 
            # compute new feature space for nonlinear separator
            poly = PolynomialFeatures(degree=2).fit_transform(X)
        else:
            poly = X

        self.classes = np.sort(np.unique(y))  # ascending order
        self._init_weights(self.classes.shape[0], X.shape[1])  # Shape of (number of classes, number of features). Every class has his own weights vector
        self._create_label_matrix(y)

        y_head = self._sigmoid(self.weights, X)  # (4,150)
        prob = self._softmax(y_head.T)  # Row vector corresponds to the probability distribution for one sample 

        self.weights = self.weights - self.tol * np.multiply((y_head - self.cls_one_hot_encoding) * X) ### ??? Shapes... weights = (3,4) - (matrix with 3,4)
        # for gd: broadcast the multiply object to one higher dimension
        loss = self._log_loss(prob)

        return loss

    def _sigmoid(self, w, X):
        Z = w @ X.T
        y_head = 1 / (1 + np.exp(-Z))
        return y_head
    
    # https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python
    def _softmax(self, y_head):
        assert len(y_head.shape) == 2
        s = np.max(y_head, axis=1)
        s = s[:, np.newaxis] # necessary step to do broadcasting. Transform array into next dimension as column vector
        e_x = np.exp(y_head - s)
        div = np.sum(e_x, axis=1)
        div = div[:, np.newaxis]
        return e_x / div

    def _gd(self):

        pass

    def _log_loss(self, prob):
        # label matrix 
        L = np.multiply(self.cls_one_hot_encoding, np.log(prob))
        log_loss = -(np.sum(L)) / len(self.cls_one_hot_encoding)
        return log_loss

    def _create_label_matrix(self, y):
        print("Classes: ", self.classes)
        self.cls_one_hot_encoding = np.zeros((len(y), len(self.classes)))
        for i, label in enumerate(y):
            self.cls_one_hot_encoding[i][label] = 1

    def _init_weights(self, num_cls, num_feat):
        self.weights = abs(np.random.randn(num_cls, num_feat))

clf = LogisticRegressionClassifier()


In [3]:
X, y = load_iris(return_X_y=True)


In [135]:
lo = clf.fit(X,y)
lo

Classes:  [0 1 2]


1.1285116393877443

In [136]:
clf.weights

array([[0.02147877, 0.18513703, 0.46064919, 1.8594102 ],
       [1.31785408, 0.47056711, 0.15139916, 1.41771074],
       [0.70883846, 1.16184783, 0.12142978, 1.26182261]])

In [57]:
A = np.random.rand(30,10)
B = np.random.rand(6,10)
C = B @ A.T
print(C.shape)
y_h = 1 / (1 + np.exp(-C))
print(y_h.shape)
print(y_h)

(6, 30)
(6, 30)
[[0.95125229 0.9504006  0.7845668  0.95517707 0.92156035 0.93280626
  0.78343239 0.96081771 0.96597665 0.9046949  0.93571762 0.93397689
  0.95119931 0.93801024 0.94121477 0.9259426  0.89389733 0.95288163
  0.91588766 0.96121476 0.88540659 0.9530323  0.91530862 0.95315505
  0.93785739 0.90473745 0.9216464  0.93598583 0.9239716  0.9414486 ]
 [0.92746087 0.89752602 0.82339353 0.92977196 0.86733657 0.90075324
  0.8258628  0.93285163 0.87542247 0.87169455 0.92934843 0.87875784
  0.9407809  0.86886707 0.88821039 0.92921688 0.89380083 0.91905964
  0.91464154 0.94505058 0.89879295 0.90794165 0.88178815 0.88395833
  0.92737879 0.90361442 0.8939087  0.84462647 0.90317036 0.90875695]
 [0.96798413 0.9517135  0.86619871 0.95992541 0.94604325 0.95360704
  0.88331074 0.97397428 0.96460837 0.93685126 0.964773   0.94165688
  0.96934114 0.95441869 0.95565049 0.95858887 0.92623808 0.96680763
  0.93919952 0.9807512  0.90939771 0.96540749 0.93375092 0.95292303
  0.95561574 0.91878258 0.9529

In [65]:
def softmax(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    #print("s max: ", s)
    s = s[:, np.newaxis] # necessary step to do broadcasting. Transform array into next dimension as column vector
    #print("s new: ", s)
    e_x = np.exp(z - s)
    #print("e_x", e_x)
    div = np.sum(e_x, axis=1)
    #print("div sum", div)
    div = div[:, np.newaxis] # dito
    #print("div", div)
    return e_x / div

x2 = np.array([[1, 2, 3, 6],  # sample 1
               [2, 4, 5, 6],  # sample 2
               [1, 2, 3, 6],
            [1, 2, 1000, 6]]) # sample 1 again(!)

sm = softmax(x2)
print(sm)
print("Summe: ",sum(sm[1,:]))

[[0.00626879 0.01704033 0.04632042 0.93037047]
 [0.01203764 0.08894682 0.24178252 0.65723302]
 [0.00626879 0.01704033 0.04632042 0.93037047]
 [0.         0.         1.         0.        ]]
Summe:  1.0


In [73]:
weights = abs(np.random.randn(3, 4))
weights.shape
weights

array([[0.98532365, 0.35034106, 0.60193241, 0.65464954],
       [0.08561235, 0.50530124, 1.2144773 , 0.53986865],
       [1.01639745, 0.18541299, 0.25126594, 0.58106588]])

In [74]:
def sigmoid(w, X):
    Z = w @ X.T
    y_head = 1 / (1 + np.exp(-Z))
    return y_head

In [45]:
x2[1][1]= 99
x2

array([[   1,    2,    3,    6],
       [   2,   99,    5,    6],
       [   1,    2,    3,    6],
       [   1,    2, 1000,    6]])

In [40]:
t = np.sort(np.array([4,2,1,4]))
we = np.zeros((len(y), 3))
we


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0

In [46]:
for i, label in enumerate(y):
    we[i][label] = 1
we

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [70]:
print(weights.shape)
print(X.shape)

(4, 4)
(150, 4)


In [79]:
we.shape

(150, 3)

In [80]:
y_head = sigmoid(weights, X)
print(y_head.shape)
prob = softmax(y_head.T)
prob.shape

(3, 150)


(150, 3)

In [86]:
T = np.array([[1,2,3], [2,2,2]])
U = np.array([[4,4,4], [1,2,3]])
print(T)
print(U)


[[2 3 4]
 [3 3 3]]
[[4 4 4]
 [1 2 3]]


In [85]:
I = np.multiply(T,U)
print(I.shape)
print(I)

(2, 3)
[[ 4  8 12]
 [ 2  4  6]]


In [115]:
s = np.sum(I, axis=1)
print(s)
s1 = np.sum(s)
s1

[24 12]


36

In [100]:
L = np.multiply(we, np.log10(prob)) + np.multiply((1 - we), (np.log10(1 - prob)))
loss_v = -(np.sum(L, axis=0) / len(X))
loss_v

array([0.27558178, 0.27569065, 0.27678485])

In [113]:
len(prob)

150

In [118]:
groundtruth = we
probability_distribution = prob

In [120]:
L = np.multiply(groundtruth, np.log(probability_distribution)) + np.multiply((1 - groundtruth), (np.log(1 - probability_distribution)))
loss_v = np.sum(L, axis=0)
loss_v1 = np.sum(loss_v)
loss_v1_final = -(loss_v1) / len(groundtruth)
loss_v1_final

1.9066723370880125

In [123]:
L_new = np.multiply(groundtruth, np.log(probability_distribution))
L_new = -(np.sum(L_new)) / len(groundtruth)
L_new

1.0966967763919089

In [125]:
l_skilearn = log_loss(y, y_head.T)
l_skilearn

1.0966768181918065

In [99]:
cost0 = we.T.dot(np.log10(prob))
cost1 = (1-we).T.dot(np.log10(1-prob))
cost = -((cost1 + cost0))/len(we) 
cost


array([[0.27558178, 0.2779912 , 0.27574518],
       [0.2768402 , 0.27569065, 0.27677651],
       [0.27688625, 0.27563625, 0.27678485]])

In [94]:
print(we.shape)
print(prob.shape)

(150, 3)
(150, 3)


In [103]:
y_head.T

array([[0.99927236, 0.98224928, 0.99816808],
       [0.99894456, 0.97687956, 0.99753863],
       [0.99872757, 0.97601703, 0.99702119],
       [0.99871071, 0.97996312, 0.99680612],
       [0.9992247 , 0.98296642, 0.99800966],
       [0.99965521, 0.99110684, 0.99896388],
       [0.99884529, 0.98155002, 0.99707615],
       [0.99921701, 0.98330444, 0.99798577],
       [0.99821232, 0.97467495, 0.99583901],
       [0.99897548, 0.97939976, 0.99750292],
       [0.99952458, 0.98609646, 0.99873028],
       [0.99910226, 0.98493351, 0.99759393],
       [0.99875671, 0.97542391, 0.99711352],
       [0.99756533, 0.96352421, 0.99483796],
       [0.99965426, 0.9834243 , 0.99913722],
       [0.9997571 , 0.99141383, 0.99926771],
       [0.99956138, 0.98562449, 0.99885446],
       [0.99931844, 0.98316648, 0.99827132],
       [0.9997163 , 0.99038441, 0.99917517],
       [0.99942222, 0.98713688, 0.99840521],
       [0.99953179, 0.98729428, 0.99872345],
       [0.99943953, 0.9871807 , 0.99846698],
       [0.

In [106]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [105]:
# verify result
loss_skilearn = log_loss(y, y_head.T)
loss_skilearn

ValueError: Found input variables with inconsistent numbers of samples: [3, 150]

In [50]:
print(we.shape)
print(prob.shape)

(150, 3)
(150, 4)


In [26]:
x1 = np.array([1, 2, 3, 4, 5])
x2 = np.array([5, 4, 3])

print(x1.shape)
x1_new = x1[:,np.newaxis]
print(x1_new, "\nShape: ", x1_new.shape)
x3 = x1_new + x2
x3

(5,)
[[1]
 [2]
 [3]
 [4]
 [5]] 
Shape:  (5, 1)


array([[ 6,  5,  4],
       [ 7,  6,  5],
       [ 8,  7,  6],
       [ 9,  8,  7],
       [10,  9,  8]])